In [17]:
import cv2
import glob
import os
import numpy as np

In [18]:
print(cv2.__version__)

4.6.0


In [19]:
kernel = np.ones((5,5),np.uint8)
kernel_big = np.ones((10,10),np.uint8)
kernel_ellipse = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))

In [20]:
def remove_objects(img, lower_size=None, upper_size=None):
  # find all objects
  nlabels, labels, stats, centroids = cv2.connectedComponentsWithStats(img)

  sizes = stats[1:, -1]
  _img = np.zeros((labels.shape))

  # process all objects, label=0 is background, objects are started from 1
  for i in range(1, nlabels):

    # remove small objects
    if (lower_size is not None) and (upper_size is not None):
      if lower_size < sizes[i - 1] and sizes[i - 1] < upper_size:
        _img[labels == i] = 255

    elif (lower_size is not None) and (upper_size is None):
      if lower_size < sizes[i - 1]:
        _img[labels == i] = 255

    elif (lower_size is None) and (upper_size is not None):
      if sizes[i - 1] < upper_size:
        _img[labels == i] = 255

  return _img

In [21]:
for file in glob.glob(os.path.join('sample', '*')):
  img = cv2.imread(file)
  gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
  ret, th = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU + cv2.THRESH_BINARY_INV)
  th = cv2.morphologyEx(th, cv2.MORPH_OPEN, kernel_ellipse)
  th = cv2.morphologyEx(th, cv2.MORPH_CLOSE, kernel_ellipse)
  th_clean = remove_objects(th, lower_size=30000, upper_size=None)
  th_clean = th_clean.astype(np.uint8)
  th_clean = cv2.morphologyEx(th_clean, cv2.MORPH_CLOSE, kernel_ellipse, iterations=2)

  # cv2.imwrite(os.path.join('binary',os.path.basename(file)), th)
  # cv2.imwrite(os.path.join('otu_clean',os.path.basename(file)), th_clean)

  th_clean_not = cv2.bitwise_not(th_clean)
  # cv2.imwrite(os.path.join('otu_clean_not',os.path.basename(file)), th_clean_not)
  th_clean_not_clean = remove_objects(th_clean_not, lower_size=None, upper_size=5000)
  th_clean_not_clean = th_clean_not_clean.astype(np.uint8)
  # cv2.imwrite(os.path.join('otu_clean_not_clean',os.path.basename(file)), th_clean_not_clean)

  anaume = cv2.bitwise_or(th_clean, th_clean_not_clean)
  cv2.imwrite(os.path.join('anaume',os.path.basename(file)), anaume)


  # 細線化(スケルトン化) THINNING_ZHANGSUEN
  skeleton1 = cv2.ximgproc.thinning(anaume, thinningType=cv2.ximgproc.THINNING_ZHANGSUEN)
  # 細線化(スケルトン化) THINNING_GUOHALL 
  skeleton2 = cv2.ximgproc.thinning(anaume, thinningType=cv2.ximgproc.THINNING_GUOHALL)

  cv2.imwrite(os.path.join('skeleton1',os.path.splitext(os.path.basename(file))[0] + '.bmp'), skeleton1)
  cv2.imwrite(os.path.join('skeleton2',os.path.splitext(os.path.basename(file))[0] + '.bmp'), skeleton2)